In [119]:
# Import of the pyomo module
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt

# Creation of a Concrete Model
model = ConcreteModel()

idx_end = 100
Dt = 0.1
model.i = Set(initialize=range(idx_end), doc='tiempo')
model.j = Set(initialize=range(idx_end-1), doc='tiempo reducido')


model.x = Var(model.i, bounds=(0.0,None), doc='current')
model.v = Var(model.i, bounds=(-5,5), doc='voltage')


def con(model, i):
    return model.x[i+1] == model.x[i] + Dt*0.5*(1/1.0*(model.v[i+1]-0.1*model.x[i+1])+1/1.0*(model.v[i]-0.1*model.x[i]))
model.demand = Constraint(model.j, rule=con, doc='restr')

def con_desired_1(model):
    return model.x[50] == 10
model.desired1 = Constraint(rule=con_desired_1, doc='restr')

def con_desired_2(model):
    return model.x[99] == 5
model.desired2 = Constraint(rule=con_desired_2, doc='restr')

def initial(model):
    return model.x[0] == 0
model.initial = Constraint(rule=initial, doc='initial value')

def objective_rule(model):
    e = 0
    for i in range(100):
        e += model.v[i]*model.x[i] + 10*model.x[i]**2
    
    return  e
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')


from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("ipopt")
results = opt.solve(model)

t = np.array([item for item in model.x.get_values()])*1.0
x = np.array([item for item in model.x.get_values().values()])
v = np.array([item for item in model.v.get_values().values()])

In [120]:
fig, (ax0,ax1) = plt.subplots(nrows=2)   # creates a figure with one axe

ax0.plot(t,x)
ax1.plot(t,v, 'g', lw=1)
#ax1.plot(t,v_1)
#ax1.plot(t,v_2)
#ax1.plot(t,v_3)

ax0.grid(True)
ax1.grid(True)

plt.show()


In [34]:
t

array([<pyomo.core.base.var._VarData object at 0x7f145aae3768>,
       <pyomo.core.base.var._VarData object at 0x7f145aae3d68>], dtype=object)

In [108]:
results

{'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Problem': [{'Number of objectives': 1, 'Number of constraints': 102, 'Number of variables': 200, 'Upper bound': inf, 'Lower bound': -inf, 'Sense': 'unknown'}], 'Solver': [{'Termination condition': 'optimal', 'Status': 'ok', 'Id': 0, 'Error rc': 0, 'Message': 'Ipopt 3.7.1\\x3a Optimal Solution Found', 'Time': 1.3075017929077148}]}